![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 2: Basic Chisel : Control Flow

#### Written by Chick Markley, Stevo Bailey, Edward Wang, and Richard Lin
[chick@berkeley.edu](mailto:chick@berkeley.edu),
[stevo@berkeley.edu](mailto:stevo@berkeley.edu),
[edwardw@berkeley.edu](mailto:edwardw@berkeley.edu), [richard.lin@berkeley.edu](mailto:edwardw@berkeley.edu)

## Introduction

Up until now there has been a strong correspondence between software and hardware in Chisel.
In control flow there will be a greater divergence between the way we think about the two.

## Table of Contents

1. [Last Connect Semantics](#last-connect)
1. [when/elsewhen/other](#when)
1. [Wire vs val and var](#wire)
1. [Exercise](#exercise)
1. [Optional Exercise ](#optional-exercise)


## Setup

First we repeat the necessary imports to run Chisel.

In [ ]:
import $ivy.`edu.berkeley.cs::chisel3:3.0-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::chisel-iotesters:1.1-SNAPSHOT_2017-07-19`
import $ivy.`edu.berkeley.cs::firrtl:1.0-SNAPSHOT_2017-07-19`
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

// A wrapper function that provides a easy way to get the generated Verilog code for a Chisel module.
// Don't worry about the implementation of this.
def getVerilog[T <: Module](gen: => T): String = {
  chisel3.Driver.execute(Array[String](), {() => gen}) match {
    case ChiselExecutionSuccess(_, _, Some(firrtl.FirrtlExecutionSuccess(_, verilog))) => verilog
  }
}

## Last Connect Semantics<a name="last-connect"></a>
 As seen earlier Chisel allows you to connect components using the ```:=``` operator.
 For various reasons it is possible to issue multiple connect statements to the same component.
 When this happens the last statement wins.  

In [ ]:
class LastConnect extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := 1.U
  io.out := 2.U
  io.out := 3.U
  io.out := 4.U
}

// Chisel Code: Declare a new tester for Simple modules
class LastConnectTester(c: LastConnect) extends PeekPokeTester(c) {
  expect(c.io.out, 4)  // Assert that the output correctly has 0
}

//  Test LastConnect
val works = Driver(() => new LastConnect) {
  c => new LastConnectTester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

## when, elsewhen, and otherwise<a name="when"></a> 
Chisel's primary implementation of conditional logic is the when, elsewhen, and otherwise constructs. 
This generally looks like
```scala
when(someBooleanCondition) {
  // things to do when true
}.elsewhen(someOtherBooleanCondition) {
  // things to do on this condition
}.otherwise {
  // things to do if none of th boolean conditions are true
}
```
They must must appear in the above order, though either of the latter may be omitted.
There can be as many elsewhen clauses as desired.
Any section that is true terminates the construct. (VERIFY THIS)
Actions taken in the bodies of the the three can be complex blocks and may contain nested
*when* and allies.
>**Unlike** scala ```if``` values are not returned by the blocks associated with when/elsewhen/otherwise.
One cannot say
```scala
val result = when(squareIt) { x * x }.otherwise { x }
```
This will **not** work. We will discuss the solution to this in the *Wires* section.


In [ ]:
/** Max3 returns the max of it's 3 arguments
  */
class Max3 extends Module {
  val io = IO(new Bundle {
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out = Output(UInt(16.W))
  })
    
  when(io.in1 > io.in2 && io.in1 > io.in3) {
    io.out := io.in1  
  }.elsewhen(io.in2 > io.in1 && io.in2 > io.in3) {
    io.out := io.in2 
  }.otherwise {
    io.out := io.in3
  }
}

// verify that the max of the three inputs is correct
class Max3Tester(c: Max3) extends PeekPokeTester(c) {
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)  
  poke(c.io.in3, 2)  
  expect(c.io.out, 6)  // input 1 should be biggest
  poke(c.io.in2, 7)  
  expect(c.io.out, 7)  // now input 2 is
  poke(c.io.in3, 11)  
  expect(c.io.out, 11) // and now input 3
  poke(c.io.in3, 3)  
  expect(c.io.out, 7)  // show that decreasing an input works as well
}

// Test Max3
val works = Driver(() => new Max3) {
  c => new Max3Tester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

## The Wire construct<a name="wire"></a>
Let's return to the note above describing the limitation that **when** does not return a value.
The Chisel **Wire** construct is one of the ways around this.
**Wire** defines a circuit component that can appear on the right hand side or left hand side of
a connect **:=** operator.


To illustrate this let's make a small combinational sorter that sorts its four numeric inputs into
its four numeric outputs. To make things clearer, consider the following graph.  Data follows the red lines
at each step when the left value is less than the right, and follows the black lines which swap the values when the left is greater than the right.
![Sort4](images/Sort4.svg)
The diagram shows a series of cells whose names begin with *row*, we will use **Wire**s to construct these as where results of a particular copy or swap are placed. 

In [ ]:
/** Sort4 sorts its 4 inputs to its 4 outputs */
class Sort4 extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(io.in0 < io.in1) {
    row10 := io.in0            // preserve first two elements
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // swap first two elements
    row11 := io.in0
  }

  when(io.in2 < io.in3) {
    row12 := io.in2            // preserve last two elements
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // swap last two elements
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(row11 < row12) {
    row21 := row11            // preserve middle 2 elements
    row22 := row12
  }.otherwise {
    row21 := row12            // swap middle two elements
    row22 := row11
  }

  when(row10 < row21) {
    io.out0 := row10            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row10
  }

  when(row10 < row21) {
    io.out0 := row10            // preserve first two elements
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // swap first two elements
    io.out1 := row10
  }

  when(row22 < row13) {
    io.out2 := row22            // preserve first two elements
    io.out3 := row13
  }.otherwise {
    io.out2 := row13            // swap first two elements
    io.out3 := row22
  }
}

// verify that the max of the three inputs is correct
class Sort4Tester(c: Sort4) extends PeekPokeTester(c) {
  poke(c.io.in0, 3)
  poke(c.io.in1, 6)
  poke(c.io.in2, 9)
  poke(c.io.in3, 12)
  expect(c.io.out0, 3)
  expect(c.io.out1, 6)
  expect(c.io.out2, 9)
  expect(c.io.out3, 12)

  poke(c.io.in0, 13)
  poke(c.io.in1, 4)
  poke(c.io.in2, 6)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)
}

// Here's the tester
val works = iotesters.Driver(() => new Sort4) {
c => new Sort4Tester(c)
}
assert(works) // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!


## Control Flow Exercise
Build Module that will compute the values of these polynomials.
- x² - 2x + 1
- 2x² + 6x + 3
- 4x² - 10x -5

Which one to compute will depenend on a selector input.  
Use **Wire**s so that hardware that computes x² appears only once in the module and so that there is a single connection to the output

First let's use test-driven development and write a model using scala

In [ ]:
def poly0(x: Int) = x*x - 2*x + 1
def poly1(x: Int) = 2*x*x + 6*x + 3
def poly2(x: Int) = 4*x*x - 10*x - 5

poly0(0)
poly1(0)
poly2(0)

poly0(1)
poly1(1)
poly2(1)

That looks ok, to make it even easier let's make a function that works like our desired hardware module.

In [ ]:
def poly(select: Int, x: Int) = {
    if(select == 0) {
        poly0(x)
    }
    else if(select == 1) {
        poly1(x)
    }
    else {
        poly2(x)
    }
}

poly(1, 0)
poly(1, 1)
poly(2, 1)



Looks like the values are correct.  So now use the following template to implement your circuit

In [ ]:
/** returns the selected result of 3 different
  * second order polynomials
  */
class Polynomial extends Module {
  val io = IO(new Bundle {
    val select = Input(UInt(4.W))
    val x = Input(SInt(32.W))
    val fOfX = Output(SInt(32.W))
  })
    
  val result = Wire(SInt(32.W))  
  val square = Wire(SInt(32.W))  
  ???

  io.fOfX := result  
}

// verify that the max of the three inputs is correct
class PolynomialTester(c: Polynomial) extends PeekPokeTester(c) {
    
  for(x <- 0 to 20) {
      for(select <- 0 to 2) {
          poke(c.io.select, select)
          poke(c.io.x, x)
          expect(c.io.fOfX, poly(select, x))

      }
  }
}

// Test Polynomial
val works = Driver(() => new Polynomial) {
  c => new PolynomialTester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!

## Control Flow Optional Exercise
Instead of having three polynomials in a single module, parameterize a Module so that you can generate hardware that computes polynomials whose co-efficients are specified at generation time. Perhaps allowing different orders of polynomials to be generated.

# Exercise answers, Don't Peek

In [ ]:
/** returns the selected result of 3 different
  * second order polynomials
  */
class Polynomial extends Module {
  val io = IO(new Bundle {
    val select = Input(UInt(4.W))
    val x = Input(SInt(32.W))
    val fOfX = Output(SInt(32.W))
  })
    
  val square = io.x * io.x
  val result = Wire(SInt(32.W))  
  when(io.select === 0.U) {
      result := (square - (2.S * io.x)) + 1.S
  }.elsewhen(io.select === 1.U) {
      result := (2.S * square) + (6.S * io.x) + 3.S
  }.otherwise {
      result := (4.S * square) - (10.S * io.x) - 5.S
  }
  io.fOfX := result  
}

// verify that the max of the three inputs is correct
class PolynomialTester(c: Polynomial) extends PeekPokeTester(c) {
    
  for(x <- 0 to 20) {
      for(select <- 0 to 2) {
          poke(c.io.select, select)
          poke(c.io.x, x)
          expect(c.io.fOfX, poly(select, x))
      }
  }
}

// Test Polynomial
val works = Driver(() => new Polynomial) {
  c => new PolynomialTester(c)
}
assert(works)        // Scala Code: if works == false, will throw an error
println("SUCCESS!!") // Scala Code: if we get here, our tests passed!